In [1]:
import os
from os import path
import numpy as np
import copy
import pandas as pd

from astropy.io import fits
import matplotlib.pyplot as plt
import matplotlib.ticker as plticker
from astropy.visualization import make_lupton_rgb
import sys

In [2]:
#Makes 3 plots per image: source simulation, lens cutout, complete simulation
def make_graphs(sim_i, sim_r, sim_g, cutout_i, cutout_r, cutout_g):
    loc = plticker.MultipleLocator(base=3.75)
    fig, (ax1, ax2, ax3) = plt.subplots(figsize=(15, 4), ncols=3)

    rgb = make_lupton_rgb(sim_i, sim_r, sim_g, Q=11., stretch=40.)
    ax1.set_title('Simulation')
    ax1.xaxis.set_ticklabels([])
    ax1.yaxis.set_ticklabels([])
    ax1.xaxis.set_major_locator(loc)
    ax1.yaxis.set_major_locator(loc)
    ax1.grid(which='major', axis='both', linestyle='-')
    original = ax1.imshow(rgb, aspect='equal')
    fig.colorbar(original, ax=ax1)

    rgb = make_lupton_rgb(cutout_i, cutout_r, cutout_g, Q=11., stretch=40.)
    ax2.set_title('Cutout')
    ax2.xaxis.set_ticklabels([])
    ax2.yaxis.set_ticklabels([])
    ax2.xaxis.set_major_locator(loc)
    ax2.yaxis.set_major_locator(loc)
    ax2.grid(which='major', axis='both', linestyle='-')
    cutout = ax2.imshow(rgb, aspect='equal')
    fig.colorbar(cutout, ax=ax2)
            
    rgb = make_lupton_rgb(sim_i+cutout_i, sim_r+cutout_r, sim_g+cutout_g, Q=11., stretch=40.)
    ax3.set_title('Complete')
    ax3.xaxis.set_ticklabels([])
    ax3.yaxis.set_ticklabels([])
    ax3.xaxis.set_major_locator(loc)
    ax3.yaxis.set_major_locator(loc)
    ax3.grid(which='major', axis='both', linestyle='-')
    projection = ax3.imshow(rgb, aspect='equal')
    fig.colorbar(projection, ax=ax3)
        
    #plt.savefig('Data/Sim_complete/Image' + new_folder + '.png', bbox_inches='tight')
    plt.show() 

In [3]:
# Picks a random sample of IDs
complete_data = pd.read_csv('Data/all_data_sim.csv')['COADD_OBJECT_ID'] 
def pick_sample(m):
    ids = complete_data.sample(n = m)
    return(ids)

# Writes fit file
def write_fit_file(name, x):
    primary = fits.PrimaryHDU()
    image = fits.ImageHDU(x, name="IMAGE")
    hdu_list = fits.HDUList([primary, image])
    hdu_list.writeto('Data/Sim_complete/' + name + '.fits', overwrite=True)

In [4]:
def complete_simulations(m, ids):
    directory_sim = '/Users/jimenagonzalez/research/DSPL/Simulations-Double-Source-Gravitational-Lensing/Data/Sim/'
    directory_cutouts = 'Data/lens_cutouts/'
    end_name = ['_g.fits', '_i.fits', '_r.fits']
    x = np.zeros((1,3,46,46)) #Complete Simulations
    n = 0
    for lens_id in ids:
        path_sim_id = directory_sim + str(lens_id)
        if(n >= m):
            break
        #check that the sim id folder exist (if not continue)
        if(os.path.exists(path_sim_id) == False):
            continue
        num_sim_id = int(len(os.listdir(path_sim_id))/3) # num sim with same lens
        #loop on each simulation produced using the same lens
        for j in range(1, num_sim_id+1):
            n += 1 #number of total simulations
            path_cutout = directory_cutouts + str(lens_id) 
            path_sim = path_sim_id + '/' + str(j)
            with fits.open(path_cutout + end_name[0]) as hdul: cutout_g = hdul[0].data
            with fits.open(path_cutout + end_name[1]) as hdul: cutout_i = hdul[0].data
            with fits.open(path_cutout + end_name[2]) as hdul: cutout_r = hdul[0].data
            with fits.open(path_sim + end_name[0]) as hdul: sim_g = hdul[0].data
            with fits.open(path_sim + end_name[1]) as hdul: sim_i = hdul[0].data
            with fits.open(path_sim + end_name[2]) as hdul: sim_r = hdul[0].data
            ele_sim = np.array([cutout_g + sim_g, cutout_r + sim_r, cutout_i + sim_i])
            x = np.append(x, [ele_sim], axis = 0)
            #make_graphs(sim_i, sim_r, sim_g, cutout_i, cutout_r, cutout_g)
    x = np.delete(x, 0, axis = 0)
    return(x) 

In [5]:
ids = pick_sample(1000)
x = complete_simulations(500, ids)
write_fit_file('complete', x)
print(len(x))

500


In [6]:
# convert -delay 140 -loop 0 *.png Complete_cut_sim.gif

In [7]:
hdu_list = fits.open('Data/Sim_complete/complete.fits')
my_images = hdu_list[1].data
print(type(my_images))
print(my_images.shape)
hdu_list.close()

<class 'numpy.ndarray'>
(500, 3, 46, 46)
